In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

Run the below cell once in a new TF 22.09 container

In [1]:
# %%bash

# cd /models && git pull && pip install .
# cd /nvtabular && git pull && pip install .
# cd /core && git pull && pip install .
# cd /systems && git pull && pip install .

# pip install tensorflow
# pip install transformers==4.21

In [3]:
# I do not need this on my machine, but can be helpful if you encounter issues

# import os
# os.environ["FORCE_TF_AVAILABLE"]="True"

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_entertainment-with-pretrained-embeddings/nvidia_logo.png" style="width: 90px; float: right;">

# Transformer-based architecture for next-item prediction task

## Overview

In this use case we will train a Transformer-based architecture for next-item prediction task.

We will use the [booking.com dataset](https://github.com/bookingcom/ml-dataset-mdt) to train a session-based model. The dataset contains 1,166,835 of anonymized hotel reservations in the train set and 378,667 in the test set. Each reservation is a part of a customer's trip (identified by `utrip_id`) which includes consecutive reservations.

We will reshape the data to organize it into 'sessions'. Each session will be a full customer itinerary in chronological order. The goal will be to predict the city_id of the final reservation of each trip.


### Learning objectives

- Training a Transformer-based architecture for next-item prediction task

## Downloading and preparing the dataset

You can download the full dataset from GitHub [here](https://github.com/bookingcom/ml-dataset-mdt). Please place it alognside this notebook (or alternatively, change the `DATAPATH` to point to where it is located).

In [2]:
from merlin.core.dispatch import get_lib
import numpy as np

DATAPATH = 'ml-dataset-mdt'

itineraries = get_lib().read_csv(f'{DATAPATH}/train_set.csv', parse_dates=['checkin'])

Each reservation has a unique `utrip_id`. During each trip a customer vists several destinations.

In [3]:
itineraries.head()

,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id
0,1000027,2016-08-13,2016-08-14,8183,desktop,7168,Elbonia,Gondal,1000027_1
1,1000027,2016-08-14,2016-08-16,15626,desktop,7168,Elbonia,Gondal,1000027_1
2,1000027,2016-08-16,2016-08-18,60902,desktop,7168,Elbonia,Gondal,1000027_1
3,1000027,2016-08-18,2016-08-21,30628,desktop,253,Elbonia,Gondal,1000027_1
4,1000033,2016-04-09,2016-04-11,38677,mobile,359,Gondal,Cobra Island,1000033_1


We will limit the sequence length to between 2 and 10 trips. That will capture upwards of 95% datapoints!

We don't want to train on trips that are shorter than two hops -- our model would not be able to learn much from such sequences. Additionally, such short sequences are uncharacteristically short for this dataset.

Besides, training on unusually long or short sequences, that are far outside of the most common sequence length, might not be the best use of our compute resources.

In [4]:
MAX_TRIP_LENGTH = 10
MIN_TRIP_LENGTH = 2

Let us now split the data into a train and validation set based on trip ID.

In [5]:
utrip_ids = itineraries.utrip_id.unique().sample(frac=1)
len(utrip_ids)

217686

In [6]:
train_set_utrip_ids = utrip_ids[:160_000]
validation_set_utrip_ids = utrip_ids[160_000:]

train_set = itineraries[itineraries.utrip_id.isin(train_set_utrip_ids)]
validation_set = itineraries[itineraries.utrip_id.isin(validation_set_utrip_ids)]

We can now begin with data preprocessing.

We will combine trips into "sessions", discard trips that are either too short or too long and calculate total trip length in stops.

We will use nvtabular for this work. It offers optimized tabular data preprocessing operators that run on the GPU. If you would like to learn more about this software library, please take a look [here](https://github.com/NVIDIA-Merlin/NVTabular).

In [7]:
from nvtabular import *
from nvtabular import ops
from merlin.models.tf import Loader

from merlin.schema.tags import Tags

2022-11-03 12:42:43.854463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 12:42:43.957475: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 12:42:44.523870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/hugectr/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/lib:/repos/dist/lib:/opt/tritonserver/lib
2022-1

2022-11-03 12:42:45.037731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 12:42:45.038256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 12:42:45.038418: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 12:42:45.061909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
train_set_dataset = Dataset(train_set)
validation_set_dataset = Dataset(validation_set)

In [9]:
groupby_features = ['city_id', 'booker_country', 'utrip_id', 'checkin'] >> ops.Groupby(
    groupby_cols=['utrip_id'],
    sort_cols=['checkin'],
    aggs={
        'city_id': ['list', 'count'],
        'booker_country': ['list']
    }
)

groupby_features_truncated_city = groupby_features['city_id_list'] >> ops.Categorify() >> ops.AddTags([Tags.SEQUENCE, Tags.ITEM, Tags.ITEM_ID])
groupby_features_truncated_country = groupby_features['booker_country_list'] >> ops.Categorify() >> ops.AddTags([Tags.SEQUENCE, Tags.ITEM])
city_id_count = groupby_features['city_id_count'] >> ops.AddTags([Tags.CONTEXT, Tags.ITEM, Tags.CONTINUOUS])

In [10]:
wf = Workflow(groupby_features_truncated_city + groupby_features_truncated_country)# + city_id_count)

In [11]:
train_set_processed = wf.fit_transform(train_set_dataset)
validation_set_processed = wf.fit_transform(validation_set_dataset)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


Our data consists of a sequence of visited `city_ids`, a sequence of `booker_countries` (represented as integer categories) and a `city_id_count` column (which contains the count of visited cities in a trip).

In [12]:
train_set_processed.compute().head()

,city_id_list,booker_country_list
0,"[7, 6, 22, 1007, 55, 60, 3]","[2, 2, 2, 2, 2, 2, 2]"
1,"[1092, 794, 138, 3]","[2, 2, 2, 2]"
2,"[3740, 275, 643, 262, 341]","[1, 1, 1, 1, 1]"
3,"[9, 370, 74, 6, 136, 280, 80, 80, 717, 9276]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
4,"[1, 368, 373, 550, 312]","[1, 1, 1, 1, 1]"


We are now ready to train our model.

In [13]:
import merlin.models.tf as mm

Let's identify two schemas. The first one for sequential features, the other for context features (`city_id_count`) that we will broadcast to the entire sequence.

In [14]:
seq_schema = train_set_processed.schema.select_by_tag(Tags.SEQUENCE)
context_schema = train_set_processed.schema.select_by_tag(Tags.CONTEXT)

Let's also identify the target column.

In [15]:
target = train_set_processed.schema.select_by_tag(Tags.SEQUENCE).column_names[0]

In [16]:
train_set_processed.compute()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


,city_id_list,booker_country_list
0,"[7, 6, 22, 1007, 55, 60, 3]","[2, 2, 2, 2, 2, 2, 2]"
1,"[1092, 794, 138, 3]","[2, 2, 2, 2]"
2,"[3740, 275, 643, 262, 341]","[1, 1, 1, 1, 1]"
3,"[9, 370, 74, 6, 136, 280, 80, 80, 717, 9276]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
4,"[1, 368, 373, 550, 312]","[1, 1, 1, 1, 1]"
...,...,...
159995,"[3, 68, 393, 143, 3]","[2, 2, 2, 2, 2]"
159996,"[179, 115, 7, 22]","[2, 2, 2, 2]"
159997,"[105, 888, 105, 105]","[1, 1, 1, 1]"
159998,"[116, 98, 116, 63]","[2, 2, 2, 2]"


In [17]:
train_set_processed.schema.select_by_tag(Tags.SEQUENCE)

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension
0,city_id_list,"(Tags.ID, Tags.SEQUENCE, Tags.ITEM, Tags.CATEG...",int64,True,True,None,0,0,0,.//categories/unique.city_id_list.parquet,0,36069,city_id_list,36070,512
1,booker_country_list,"(Tags.SEQUENCE, Tags.ITEM, Tags.CATEGORICAL)",int64,True,True,None,0,0,0,.//categories/unique.booker_country_list.parquet,0,5,booker_country_list,6,16


In [18]:
train_set_processed.schema.select_by_tag(Tags.CONTEXT)

""


In [19]:
loader = Loader(train_set_processed, batch_size=1024, shuffle=True)

In [20]:
# next(iter(loader))

In [21]:
from merlin.models.tf.transforms.features import BroadcastToSequence

In [22]:
context_schema

""


In [23]:
seq_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension
0,city_id_list,"(Tags.ID, Tags.SEQUENCE, Tags.ITEM, Tags.CATEG...",int64,True,True,None,0,0,0,.//categories/unique.city_id_list.parquet,0,36069,city_id_list,36070,512
1,booker_country_list,"(Tags.SEQUENCE, Tags.ITEM, Tags.CATEGORICAL)",int64,True,True,None,0,0,0,.//categories/unique.booker_country_list.parquet,0,5,booker_country_list,6,16


## Without broadcasting of context features

In [26]:
train_set_processed.compute()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


,city_id_list,booker_country_list
0,"[7, 6, 22, 1007, 55, 60, 3]","[2, 2, 2, 2, 2, 2, 2]"
1,"[1092, 794, 138, 3]","[2, 2, 2, 2]"
2,"[3740, 275, 643, 262, 341]","[1, 1, 1, 1, 1]"
3,"[9, 370, 74, 6, 136, 280, 80, 80, 717, 9276]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
4,"[1, 368, 373, 550, 312]","[1, 1, 1, 1, 1]"
...,...,...
159995,"[3, 68, 393, 143, 3]","[2, 2, 2, 2, 2]"
159996,"[179, 115, 7, 22]","[2, 2, 2, 2]"
159997,"[105, 888, 105, 105]","[1, 1, 1, 1]"
159998,"[116, 98, 116, 63]","[2, 2, 2, 2]"


In [24]:
model = mm.Model(
    mm.InputBlockV2(
        seq_schema,
        embeddings=mm.Embeddings(
            train_set_processed.schema.select_by_tag(Tags.CATEGORICAL), sequence_combiner=None
        ),
    ),

    mm.GPT2Block(d_model=40, n_head=4, n_layer=2, pre=mm.ReplaceMaskedEmbeddings()),
    mm.CategoricalOutput(
        train_set_processed.schema.select_by_name(target),
        default_loss="categorical_crossentropy",
    ),
)

In [25]:
model.compile(run_eagerly=True, optimizer='adam', loss="categorical_crossentropy")
model.fit(loader, pre=mm.SequenceMaskRandom(schema=seq_schema, target=target, masking_prob=0.3))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: 4, 4, 4, ...

## Specifying correct input_dimensions (before broadcasting) in model constructor (d_model=40)

In [ ]:
model = mm.Model(
    mm.InputBlockV2(
        train_set_processed.schema,
        embeddings=mm.Embeddings(
            train_set_processed.schema.select_by_tag(Tags.CATEGORICAL), sequence_combiner=None
        ),
        post=BroadcastToSequence(context_schema, seq_schema)
    ),

    mm.GPT2Block(d_model=40, n_head=4, n_layer=2, pre=mm.ReplaceMaskedEmbeddings()),
    mm.CategoricalOutput(
        train_set_processed.schema.select_by_name(target),
        default_loss="categorical_crossentropy",
    ),
)

In [ ]:
model.compile(run_eagerly=True, optimizer='adam', loss="categorical_crossentropy")
model.fit(loader, pre=mm.SequenceMaskRandom(schema=train_set_processed.schema, target=target, masking_prob=0.3))

## Specifying correct input_dimensions (after broadcasting) in model constructor (d_model=41)

In [ ]:
model = mm.Model(
    mm.InputBlockV2(

        train_set_processed.schema,
        embeddings=mm.Embeddings(
            train_set_processed.schema.select_by_tag(Tags.CATEGORICAL), sequence_combiner=None
        ),
        post=BroadcastToSequence(context_schema, seq_schema)
    ),

    mm.GPT2Block(d_model=41, n_head=4, n_layer=2, pre=mm.ReplaceMaskedEmbeddings()),
    mm.CategoricalOutput(
        train_set_processed.schema.select_by_name(target),
        default_loss="categorical_crossentropy",
    ),
)

In [ ]:
model.compile(run_eagerly=True, optimizer='adam', loss="categorical_crossentropy")
model.fit(loader, pre=mm.SequenceMaskRandom(schema=train_set_processed.schema, target=target, masking_prob=0.3))

In [ ]:
# first stab at evaluation

loader_eval = Loader(validation_set_processed, batch_size=1024, shuffle=False)

In [ ]:
model.evaluate(loader_eval, batch_size=1024, pre=mm.SequenceMaskLast(schema=train_set_processed.schema, target=target))